In [10]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
# from cuml import SVC

In [11]:
data_path = "./Data.csv"
df = pd.read_csv(data_path)

# df = df[['networkExp', 'bedwars_level', 'Experience', 'beds_broken_bedwars', 'beds_lost_bedwars', 'coins', 'deaths_bedwars', 'diamond_resources_collected_bedwars', 'emerald_resources_collected_bedwars', 'final_deaths_bedwars', 'final_kills_bedwars', 'games_played_bedwars', 'games_played_bedwars_1', 'kills_bedwars', 'losses_bedwars', 'void_final_deaths_bedwars',  'wins_bedwars','fkdr','wlr','bblr','fk_lev','bb_lev','kill_lev', "Cheat"]]
df = df[['karma', 'bedwars_level',
        'bedwars_loot_box',
        'all_timeBEDWARS__defensive', 'BEDWARS__offensive',
        'BEDWARS__support', 'Bedwars_openedChests',
        'beds_broken_bedwars', 'beds_lost_bedwars',
        'deaths_bedwars', 'diamond_resources_collected_bedwars',
        'emerald_resources_collected_bedwars', 'fall_deaths_bedwars',
        'final_deaths_bedwars', 'final_kills_bedwars',
        'games_played_bedwars', 'games_played_bedwars_1', 'kills_bedwars',
        'losses_bedwars', 'void_deaths_bedwars', 'void_final_deaths_bedwars',
        'void_final_kills_bedwars', 'void_kills_bedwars', 'wins_bedwars', "Cheat"
        ]]

In [12]:
# Cheatカラムを削除して、各レコードの合計を計算する
row_sums = df.drop('Cheat', axis=1).sum(axis=1)

# 分母が0になる可能性がある行のインデックスを取得
zero_division_indices = row_sums[row_sums == 0].index

# 10,000をその行の合計値で割った値を計算して保持する
divisors = pd.Series(0, index=df.index)  # 全ての行を0で初期化
divisors[zero_division_indices] = 0  # 分母が0になる行のみ0に設定
divisors[~divisors.index.isin(zero_division_indices)] = 10000 / row_sums[~row_sums.index.isin(zero_division_indices)]

# それぞれの行に対して計算した値を掛ける
df_scaled = df.drop('Cheat', axis=1).mul(divisors, axis=0)

# データフレームにCheatカラムを追加する
df_scaled['Cheat'] = df['Cheat']
df_scaled

# 新しいデータベースとして保存する
# df_scaled.to_csv('ScaledData.csv', index=False)

/var/folders/nz/pdh3wbl139nfbnd6j9xbwx300000gn/T/ipykernel_28246/598129436.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.02585146 1.68378515 0.12600965 ... 1.43472023 0.49970018 3.87747189]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  divisors[~divisors.index.isin(zero_division_indices)] = 10000 / row_sums[~row_sums.index.isin(zero_division_indices)]


,karma,bedwars_level,bedwars_loot_box,all_timeBEDWARS__defensive,BEDWARS__offensive,BEDWARS__support,Bedwars_openedChests,beds_broken_bedwars,beds_lost_bedwars,deaths_bedwars,...,games_played_bedwars,games_played_bedwars_1,kills_bedwars,losses_bedwars,void_deaths_bedwars,void_final_deaths_bedwars,void_final_kills_bedwars,void_kills_bedwars,wins_bedwars,Cheat
0,5985.843250,10.258515,1.025851,0.0,55.395979,2.051703,3.077554,215.428806,167.213787,825.810423,...,270.824785,270.824785,414.443989,254.411161,385.720148,57.447682,110.791957,204.144440,16.413623,0
1,75.770332,23.572992,26.940562,0.0,15.154066,25.256777,75.770332,195.319077,616.265365,1655.160801,...,666.778919,666.778919,720.660044,646.573497,939.552113,249.200202,126.283886,328.338104,20.205422,0
2,9702.113182,0.882068,0.630048,0.0,2.142164,1.008077,1.386106,7.938608,12.096927,50.403861,...,19.279477,19.405486,30.998374,15.751207,24.571882,1.764135,3.906299,12.852985,2.646203,0
3,5869.074492,12.899065,0.000000,0.0,48.371493,16.123831,0.000000,238.632699,125.765882,764.269590,...,203.160271,203.160271,290.228958,164.463076,412.770074,51.596259,96.742986,103.192519,38.697194,0
4,0.000000,22.994379,8.942259,0.0,5.109862,93.254982,21.716914,71.538068,450.945324,1209.759836,...,500.766479,512.263669,937.659683,444.557997,657.894737,172.457844,29.381707,301.481860,31.936638,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4673,2476.176620,0.727431,0.000000,0.0,34.189278,20.368080,0.000000,217.502000,0.727431,1291.190805,...,404.451880,504.837419,1419.946170,0.727431,608.860115,66.196261,185.495017,688.150142,189.132174,1
4674,30.487805,1.219512,3.658537,0.0,3.658537,29.268293,0.000000,43.902439,1.219512,1828.048780,...,809.756098,904.878049,1210.975610,1.219512,1093.902439,270.731707,46.341463,435.365854,164.634146,1
4675,344.332855,1.434720,2.869440,0.0,0.000000,30.129125,0.000000,20.086083,1.434720,2314.203730,...,923.959828,1114.777618,992.826399,1.434720,1439.024390,449.067432,64.562410,388.809182,113.342898,1
4676,117.429542,0.499700,3.497901,0.0,4.497302,17.489506,0.000000,116.929842,0.499700,2604.937038,...,1103.337997,1463.122127,320.307815,0.499700,1615.530682,478.213072,50.469718,170.897462,78.952628,1


In [13]:
# df = pd.get_dummies(df)
# print("dummies")

X = df_scaled.drop('Cheat', axis=1)
y = df_scaled['Cheat']
df_scaled

# データの標準化
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)


# 不足しているデータのインデックスを特定
underrepresented_indices = df_scaled.index

missing_indices = [index for index in underrepresented_indices if index not in X_train.index]
print("Missing indices:", missing_indices)

# Filter out the missing indices
underrepresented_indices = [index for index in underrepresented_indices if index in X_train.index]
X_underrepresented = X_train.loc[underrepresented_indices]
y_underrepresented = y_train.loc[underrepresented_indices]


# 不足しているデータだけを抽出
X_underrepresented = X_train.loc[underrepresented_indices]
y_underrepresented = y_train.loc[underrepresented_indices]

# SMOTEでオーバーサンプリングを実行
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_underrepresented, y_underrepresented)

# オーバーサンプリング後のデータを元のデータに結合
X_train_resampled = pd.concat([X_train, X_resampled])
y_train_resampled = pd.concat([y_train, y_resampled])

# モデルの作成と訓練（オーバーサンプリング後のデータを使用）
model = SVC(random_state=42, class_weight={0: 1, 1:10}, kernel="linear")
model.fit(X_train_resampled, y_train_resampled)



# テストデータを使って予測を行う
y_pred = model.predict(X_test)


# # モデルの作成
# model = RandomForestClassifier(n_estimators=200, random_state=42)
# model.fit(X_train, y_train)


# # モデルの訓練
# model.fit(X_train, y_train)

# # テストデータを使って予測を行う
# y_pred = model.predict(X_test)

# 正解率を計算する
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# 適合率を計算する
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# 再現率を計算する
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1スコアを計算する
f1 = f1_score(y_test, y_pred)
print(f'F1 Score: {f1}')

# 混同行列を表示する
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# モデルと標準化器の保存
# joblib.dump(model, './models/Cheater.pkl')
# joblib.dump(scaler, './models/scaler.joblib')


Missing indices: [39, 42, 45, 49, 57, 72, 98, 104, 113, 117, 124, 130, 134, 138, 148, 154, 159, 162, 170, 173, 189, 192, 214, 218, 220, 223, 224, 246, 269, 271, 291, 294, 302, 304, 308, 348, 379, 383, 394, 398, 402, 405, 410, 415, 453, 472, 481, 483, 489, 491, 496, 500, 502, 514, 517, 521, 526, 530, 531, 538, 541, 543, 545, 556, 562, 567, 578, 582, 584, 599, 634, 636, 639, 651, 668, 675, 684, 716, 727, 731, 776, 780, 788, 812, 825, 829, 847, 878, 883, 896, 918, 927, 933, 938, 962, 965, 980, 991, 1001, 1003, 1011, 1014, 1042, 1051, 1065, 1066, 1077, 1085, 1091, 1098, 1114, 1118, 1161, 1170, 1185, 1196, 1202, 1220, 1223, 1232, 1261, 1267, 1271, 1275, 1298, 1319, 1326, 1330, 1338, 1351, 1357, 1362, 1366, 1376, 1377, 1382, 1385, 1389, 1410, 1412, 1423, 1450, 1452, 1470, 1475, 1487, 1490, 1509, 1511, 1513, 1515, 1522, 1525, 1530, 1531, 1556, 1563, 1564, 1567, 1571, 1586, 1620, 1621, 1632, 1657, 1662, 1675, 1680, 1698, 1705, 1708, 1710, 1742, 1759, 1761, 1767, 1778, 1793, 1828, 1830, 1831, 1

In [ ]:
joblib.dump(model, '../Cheater.pkl')
# joblib.dump(scaler, './models/scaler.joblib')

['../Cheater.pkl']

# snipe垢の条件

fkdr, wlr, bblrは低い

star n.levは30以下

sniper%が高い　（ターゲットが見つかるまで何回もｒｑするから）

fk/lev bb/lev kill/lev が高い　fk/levは7.5 bb/levは6.5 kill/levは20以上で高い

# cheatersデータ - snipe垢 = not sniper cheaters
snipe垢の条件外のcheaterを予測する。

# cubelifyのsniper%
cubelifyのsniper%を取得して、学習データとする